---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [156]:
def date_sorter():
    import re
    
    df_dates = pd.DataFrame([])

    # Your code here
    pat1 = re.compile(r"((?P<mon>(?:[0-9]|[0-1][0-9]))[\/-](?P<day>(?:[1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1]))[\/-](?P<year>(?:(?:(?:19|20)[0-9][0-9]|[0-9][0-9]))))")
    pat2 = re.compile(r"((?P<day>(?:[1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1])) (?P<mon>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z|\s|.|,]+(?P<year>(?:(?:(?:19|20)[0-9][0-9]|[0-9][0-9]))))")
    pat3 = re.compile(r"((?P<mon>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z-\s.,]+(?P<day>(?:[1-9]|0[1-9]|1[0-9]|2[0-9]|3[0-1]))[,|.|\s|t|h|n|d|s|t|-]+(?P<year>(?:(?:(?:19|20)[0-9][0-9]|[0-9][0-9]))))")
    pat4 = re.compile(r"((?P<mon>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z|\s|.|,]+(?P<year>(?:(?:19|20)[0-9][0-9])))")
    pat5 = re.compile(r"((?P<mon>(?:[1-9]|[0-1][0-9]))[\/](?P<year>(?:(?:19|20)[0-9][0-9])))")
    pat6 = re.compile(r"((?P<year>(?:(?:19|20)[0-9][0-9])))")
    
    df_dates1 = df.str.extractall(pat1)
    df_dates2 = df.str.extractall(pat2)
    df_dates = pd.concat([df_dates1, df_dates2])
    
    df_dates3 = df.str.extractall(pat3)
    df_dates = pd.concat([df_dates, df_dates3])
    
    df_dates4 = df.str.extractall(pat4)
    df_dates4['day'] = '1'
    df_dates = pd.concat([df_dates, df_dates4])
    df_dates = df_dates[~df_dates.index.duplicated(keep='first')]
    
    df_dates5 = df.str.extractall(pat5)
    df_dates5['day'] = '1'
    df_dates = pd.concat([df_dates, df_dates5])
    df_dates = df_dates[~df_dates.index.duplicated(keep='first')]
    
    df_dates6 = df.str.extractall(pat6)
    df_dates6['day'] = '1'
    df_dates6['mon'] = '1'
    df_dates = pd.concat([df_dates, df_dates6])
    df_dates = df_dates[~df_dates.index.duplicated(keep='first')]
    
    df_dates['day'] = df_dates.day.astype(int)

    import calendar
    abbr_to_num = {name: num for num, name in enumerate(calendar.month_abbr) if num}
    
    df_dates['mon'] = df_dates['mon'].apply(lambda x: abbr_to_num[x] if x in list(calendar.month_abbr) else int(x))
    df_dates['year'] = df_dates.year.astype(int)
    df_dates['year'] = df_dates.year.apply(lambda x: x + 1900 if x < 100 else x)
    
#     df_dates['date'] = 
        
    return df_dates # Your answer here

df_dates = date_sorter()

df_dates



,,0,mon,day,year
,match,,,,
0,0,03/25/93,3,25,1993
1,0,6/18/85,6,18,1985
2,0,7/8/71,7,8,1971
3,0,9/27/75,9,27,1975
4,0,2/6/96,2,6,1996
5,0,7/06/79,7,6,1979
6,0,5/18/78,5,18,1978
7,0,10/24/89,10,24,1989
8,0,3/7/86,3,7,1986
